In [1]:
import pandas as pd
from sequenzo import *

In [2]:
# ------------------------
# Step 0: 定义提取工具函数
# ------------------------

def extract_state_space(sequences):
    return sorted(set(s for seq in sequences for s in seq))

# ------------------------
# Step 1: 读取和准备数据
# ------------------------

df = pd.read_csv("/Users/lei/Documents/Sequenzo_all_folders/sequenzo_local/test_data/real_data_my_paper/detailed_sequence_10_work_years_df.csv")

time_cols = [f"C{i}" for i in range(1, 11)]  # C1~C10

# 假设 'country' 是一列，值为 'India' 或 'US'
india_df = df[df["country"] == "india"].copy()
us_df = df[df["country"] == "us"].copy()

T = len(time_cols)  # 一般是固定 10

# 提取 sequence 列
india_sequences = extract_sequences(india_df, time_cols)
us_sequences = extract_sequences(us_df, time_cols)

india_states = extract_state_space(india_sequences)
us_states = extract_state_space(us_sequences)

# ------------------------
# Step 2: 构建前缀树
# ------------------------

india_tree = build_prefix_tree(india_sequences)
us_tree = build_prefix_tree(us_sequences)

# 为每个国家分别建模
india_model = IndividualDivergence(india_sequences)
us_model = IndividualDivergence(us_sequences)

In [3]:
df

,worker_id,gender,country,cohort,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10
0,dilip-kumar-nath-7538746a,male,india,11-20 total work years,software,software,software,software,software,software,software,software,software,software
1,vinodkumar-yadav-b7a976107,male,india,11-20 total work years,software,software,software,software,software,software,software,software,software,software
2,josh-slosson-07a1509,male,india,11-20 total work years,software,software,software,software,software,software,software,software,software,software
3,jonatthan-rodriguez-8a5612121,male,india,11-20 total work years,software,software,software,software,software,software,software,software,software,software
4,jeenesh-jyothindran-8ba28818,male,india,11-20 total work years,software,software,software,software,software,software,software,software,software,software
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38897,nan-jiang-743a9863,female,us,0-10 total work years,software,software,software,software,software,software,software,software,software,software
38898,tina-pava-4846207,female,us,21-30 total work years,software,software,software,software,software,software,software,software,software,software
38899,sariandoni,female,us,21-30 total work years,support & test,software,software,software,software,software,software,research,research,research
38900,nancy-jain-a8723460,female,us,0-10 total work years,software,software,software,software,software,support & test,software,software,software,software


计算每年 prefix rarity

如果某些年几乎所有人相同（方差接近 0），z 分数会是 NaN；这种情况下原始尺度的 rarity 更稳健。


In [4]:
# 返回 DataFrame，列名 t1..t10；zscore=True 则按年做列内标准化
df_rarity_india = india_model.compute_prefix_rarity_per_year(as_dataframe=True, column_prefix="t", zscore=False)
df_rarity_us = us_model.compute_prefix_rarity_per_year(as_dataframe=True, column_prefix="t", zscore=False)
# 若需要 z 分数版本：
# df_rarity_z = div.compute_prefix_rarity_per_year(as_dataframe=True, column_prefix="t", zscore=True)

df_rarity_india

,t1,t2,t3,t4,t5,t6,t7,t8,t9,t10
0,0.499509,0.517442,0.557534,0.622334,0.693213,0.750123,0.795696,0.837312,0.881535,0.915332
1,0.499509,0.517442,0.557534,0.622334,0.693213,0.750123,0.795696,0.837312,0.881535,0.915332
2,0.499509,0.517442,0.557534,0.622334,0.693213,0.750123,0.795696,0.837312,0.881535,0.915332
3,0.499509,0.517442,0.557534,0.622334,0.693213,0.750123,0.795696,0.837312,0.881535,0.915332
4,0.499509,0.517442,0.557534,0.622334,0.693213,0.750123,0.795696,0.837312,0.881535,0.915332
...,...,...,...,...,...,...,...,...,...,...
15108,0.499509,0.517442,0.557534,0.622334,0.693213,0.750123,0.795696,0.837312,0.881535,0.915332
15109,3.272425,3.341044,3.412710,5.959749,6.012393,6.126803,8.524698,8.524698,9.623309,9.623309
15110,0.499509,0.517442,0.557534,0.622334,0.693213,0.750123,0.795696,0.837312,0.881535,0.915332
15111,2.091222,2.133340,2.199742,2.273437,2.337119,2.397102,2.451654,2.493212,2.519989,2.559407


In [5]:
# 已有：
# df_rarity_india = india_model.compute_prefix_rarity_per_year(as_dataframe=True, column_prefix="t", zscore=False)
# df_rarity_us    = us_model.compute_prefix_rarity_per_year(as_dataframe=True, column_prefix="t", zscore=False)

# 若你有对应的 worker_id 列表（顺序与 model.sequences 完全对齐）：
# 例如若你当时用 df_india_seq（含 worker_id，行顺序与 sequences 一致）建立了 india_model：
# 注意：这个df是最开始引入的df，没有经过任何数据处理的df
df_india = df[df['country'] == 'india']
df_us = df[df['country'] == 'us']

worker_ids_india = df_india["worker_id"].tolist()
worker_ids_us    = df_us["worker_id"].tolist() 

# 校验长度
assert len(worker_ids_india) == df_rarity_india.shape[0], \
    f"India length mismatch: ids={len(worker_ids_india)} rarity={df_rarity_india.shape[0]}"

assert len(worker_ids_us) == df_rarity_us.shape[0], \
    f"US length mismatch: ids={len(worker_ids_us)} rarity={df_rarity_us.shape[0]}"

def wide_to_long_rarity(df_rarity_wide: pd.DataFrame, worker_ids, country_name: str) -> pd.DataFrame:
    out = df_rarity_wide.copy()
    out["worker_id"] = worker_ids
    out["country"] = country_name
    long_df = out.melt(
        id_vars=["worker_id", "country"],
        var_name="year",
        value_name="rarity_score"
    )
    # t1 -> 1, t2 -> 2, ...
    long_df["year"] = long_df["year"].str.replace("t", "", regex=False).astype(int)
    return long_df

df_long_india = wide_to_long_rarity(df_rarity_india, worker_ids_india, "India")
df_long_us    = wide_to_long_rarity(df_rarity_us,    worker_ids_us,    "US")

# 合并两个国家
df_rarity_long = pd.concat([df_long_india, df_long_us], ignore_index=True)

# 可选：排序、重置索引
df_rarity_long = df_rarity_long.sort_values(["country", "worker_id", "year"]).reset_index(drop=True)

df_rarity_long
# df_rarity_long 就是你要的长表：
# 列为：worker_id, country, year, rarity_score

,worker_id,country,year,rarity_score
0,%7E%7E%7Eabhishek%7E%7E%7E-lifoline-4ba22719,India,1,0.499509
1,%7E%7E%7Eabhishek%7E%7E%7E-lifoline-4ba22719,India,2,0.517442
2,%7E%7E%7Eabhishek%7E%7E%7E-lifoline-4ba22719,India,3,0.557534
3,%7E%7E%7Eabhishek%7E%7E%7E-lifoline-4ba22719,India,4,0.622334
4,%7E%7E%7Eabhishek%7E%7E%7E-lifoline-4ba22719,India,5,0.693213
...,...,...,...,...
389015,zwheeler,US,6,0.470416
389016,zwheeler,US,7,0.486217
389017,zwheeler,US,8,0.499013
389018,zwheeler,US,9,0.511134


In [6]:
df_seniority = pd.read_csv("/Users/lei/Documents/Sequenzo_all_folders/sequenzo_local/test_data/real_data_my_paper/detailed_senority_levels_for_10_years.csv")
df_seniority

,worker_id,gender,country,cohort,highest_educational_degree,whether_bachelor_university_prestigious,internationalization,work_years,company_size,Y1,Y2,Y3,Y4,Y5,Y6,Y7,Y8,Y9,Y10
0,dilip-kumar-nath-7538746a,male,india,11-20 total work years,Bachelor,False,Multinational,15.250,"1,001-5,000 employees",Regular,Regular,Regular,Regular,Senior,Senior,Senior,Senior,Senior,Senior
1,vinodkumar-yadav-b7a976107,male,india,11-20 total work years,Bachelor,False,Local,11.500,"5,001-10,000 employees",Regular,Regular,Regular,Regular,Regular,Regular,Regular,Regular,Regular,Regular
2,rajani-kulkarni-72674649,female,india,11-20 total work years,Bachelor,False,Multinational,11.250,"10,001+ employees",Regular,Regular,Regular,Regular,Regular,Regular,Regular,Leader,Leader,Regular
3,josh-slosson-07a1509,male,india,11-20 total work years,Bachelor,False,International,18.417,11-50 employees,Regular,Regular,Regular,Regular,Regular,Regular,Regular,Regular,Regular,Regular
4,jonatthan-rodriguez-8a5612121,male,india,11-20 total work years,Bachelor,False,Multinational,11.917,"5,001-10,000 employees",Regular,Regular,Senior,Senior,Senior,Senior,Senior,Senior,Senior,Senior
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32623,edwardgu1915,male,us,0-10 total work years,Master,False,Multinational,10.499,"501-1,000 employees",Leader,Leader,Leader,Leader,Leader,Senior,Senior,Senior,Regular,Senior
32624,terry-ferguson-8060363a,male,us,21-30 total work years,Bachelor,False,Local,21.001,"10,001+ employees",Regular,Regular,Regular,Regular,Regular,Regular,Regular,Regular,Regular,Regular
32625,christy-garner-677894213,female,us,21-30 total work years,Bachelor,False,Multinational,27.250,"501-1,000 employees",Regular,Regular,Regular,Regular,Regular,Regular,Regular,Regular,Regular,Regular
32626,jay-p-119369202,male,us,21-30 total work years,Bachelor,False,Local,23.417,"10,001+ employees",Regular,Regular,Regular,Regular,Regular,Regular,Regular,Regular,Regular,Regular


In [7]:
# 将 df_seniority 从宽数据转换为长数据
def wide_to_long_seniority(df_wide: pd.DataFrame) -> pd.DataFrame:
    """
    将宽格式的 seniority 数据转换为长格式
    输出列: worker_id, country, year, seniority
    """
    # 找出时间列（假设是以 Y 开头的列，如 Y1, Y2, ..., Y10）
    time_cols = [col for col in df_wide.columns if col.startswith('Y') and col[1:].isdigit()]
    
    print(f"检测到的时间列: {time_cols}")
    
    # 转换为长格式 - 如果有country列则保留
    id_vars = ['worker_id']
    if 'country' in df_wide.columns:
        id_vars.append('country')
    
    df_long = df_wide.melt(
        id_vars=id_vars,
        value_vars=time_cols,
        var_name='year',
        value_name='seniority'
    )
    
    # 如果年份列是 Y1, Y2 格式，转换为数字
    if df_long['year'].iloc[0].startswith('Y'):
        df_long['year'] = df_long['year'].str.replace('Y', '', regex=False).astype(int)
    
    # 排序并重置索引
    sort_cols = ['worker_id', 'year']
    if 'country' in df_long.columns:
        sort_cols = ['country', 'worker_id', 'year']
    df_long = df_long.sort_values(sort_cols).reset_index(drop=True)
    
    return df_long

# 执行转换
df_seniority_long = wide_to_long_seniority(df_seniority)

print("转换后的长数据格式:")
print(f"形状: {df_seniority_long.shape}")
print(f"列名: {df_seniority_long.columns.tolist()}")
print("\n前10行数据:")
df_seniority_long.head(10)


检测到的时间列: ['Y1', 'Y2', 'Y3', 'Y4', 'Y5', 'Y6', 'Y7', 'Y8', 'Y9', 'Y10']
转换后的长数据格式:
形状: (326280, 4)
列名: ['worker_id', 'country', 'year', 'seniority']

前10行数据:


,worker_id,country,year,seniority
0,%E2%97%8F%E0%B9%8B-b%C3%A1%DA%A9t%C3%BD%C3%A1v...,india,1,Senior
1,%E2%97%8F%E0%B9%8B-b%C3%A1%DA%A9t%C3%BD%C3%A1v...,india,2,Senior
2,%E2%97%8F%E0%B9%8B-b%C3%A1%DA%A9t%C3%BD%C3%A1v...,india,3,Senior
3,%E2%97%8F%E0%B9%8B-b%C3%A1%DA%A9t%C3%BD%C3%A1v...,india,4,Senior
4,%E2%97%8F%E0%B9%8B-b%C3%A1%DA%A9t%C3%BD%C3%A1v...,india,5,Senior
5,%E2%97%8F%E0%B9%8B-b%C3%A1%DA%A9t%C3%BD%C3%A1v...,india,6,Senior
6,%E2%97%8F%E0%B9%8B-b%C3%A1%DA%A9t%C3%BD%C3%A1v...,india,7,Senior
7,%E2%97%8F%E0%B9%8B-b%C3%A1%DA%A9t%C3%BD%C3%A1v...,india,8,Senior
8,%E2%97%8F%E0%B9%8B-b%C3%A1%DA%A9t%C3%BD%C3%A1v...,india,9,Senior
9,%E2%97%8F%E0%B9%8B-b%C3%A1%DA%A9t%C3%BD%C3%A1v...,india,10,Senior


In [8]:
# 重新执行转换以确保包含country列
df_seniority_long = wide_to_long_seniority(df_seniority)

print("重新转换后的长数据格式:")
print(f"形状: {df_seniority_long.shape}")
print(f"列名: {df_seniority_long.columns.tolist()}")
print("\n前5行数据:")
print(df_seniority_long.head())


检测到的时间列: ['Y1', 'Y2', 'Y3', 'Y4', 'Y5', 'Y6', 'Y7', 'Y8', 'Y9', 'Y10']
重新转换后的长数据格式:
形状: (326280, 4)
列名: ['worker_id', 'country', 'year', 'seniority']

前5行数据:
                                           worker_id country  year seniority
0  %E2%97%8F%E0%B9%8B-b%C3%A1%DA%A9t%C3%BD%C3%A1v...   india     1    Senior
1  %E2%97%8F%E0%B9%8B-b%C3%A1%DA%A9t%C3%BD%C3%A1v...   india     2    Senior
2  %E2%97%8F%E0%B9%8B-b%C3%A1%DA%A9t%C3%BD%C3%A1v...   india     3    Senior
3  %E2%97%8F%E0%B9%8B-b%C3%A1%DA%A9t%C3%BD%C3%A1v...   india     4    Senior
4  %E2%97%8F%E0%B9%8B-b%C3%A1%DA%A9t%C3%BD%C3%A1v...   india     5    Senior


In [9]:
# 合并 df_rarity_long 和 df_seniority_long，以前者为准（左连接）
print("合并前的数据情况:")
print(f"df_rarity_long 形状: {df_rarity_long.shape}")
print(f"df_rarity_long 列名: {df_rarity_long.columns.tolist()}")
print(f"df_seniority_long 形状: {df_seniority_long.shape}")
print(f"df_seniority_long 列名: {df_seniority_long.columns.tolist()}")

# 确定合并的键
merge_keys = ['worker_id', 'year']
if 'country' in df_seniority_long.columns:
    merge_keys = ['worker_id', 'country', 'year']

print(f"\n使用的合并键: {merge_keys}")

# 执行左连接
df_merged = df_rarity_long.merge(
    df_seniority_long,
    on=merge_keys,
    how='left'
)

print(f"\n合并后的数据:")
print(f"形状: {df_merged.shape}")
print(f"列名: {df_merged.columns.tolist()}")
print(f"\n前10行数据:")
print(df_merged.head(10))

# 检查是否有匹配失败的记录
missing_seniority = df_merged['seniority'].isnull().sum()
print(f"\n缺失 seniority 数据的记录数: {missing_seniority}")
if missing_seniority > 0:
    print(f"缺失比例: {missing_seniority / len(df_merged) * 100:.2f}%")


合并前的数据情况:
df_rarity_long 形状: (389020, 4)
df_rarity_long 列名: ['worker_id', 'country', 'year', 'rarity_score']
df_seniority_long 形状: (326280, 4)
df_seniority_long 列名: ['worker_id', 'country', 'year', 'seniority']

使用的合并键: ['worker_id', 'country', 'year']

合并后的数据:
形状: (389020, 5)
列名: ['worker_id', 'country', 'year', 'rarity_score', 'seniority']

前10行数据:
                                      worker_id country  year  rarity_score  \
0  %7E%7E%7Eabhishek%7E%7E%7E-lifoline-4ba22719   India     1      0.499509   
1  %7E%7E%7Eabhishek%7E%7E%7E-lifoline-4ba22719   India     2      0.517442   
2  %7E%7E%7Eabhishek%7E%7E%7E-lifoline-4ba22719   India     3      0.557534   
3  %7E%7E%7Eabhishek%7E%7E%7E-lifoline-4ba22719   India     4      0.622334   
4  %7E%7E%7Eabhishek%7E%7E%7E-lifoline-4ba22719   India     5      0.693213   
5  %7E%7E%7Eabhishek%7E%7E%7E-lifoline-4ba22719   India     6      0.750123   
6  %7E%7E%7Eabhishek%7E%7E%7E-lifoline-4ba22719   India     7      0.795696   
7  %7E%7E%7Eabh

In [10]:
# 详细调查缺失数据的原因
print("=== 调查缺失 seniority 数据的原因 ===")

# 1. 检查两个数据框的基本信息
print("\n1. 基本信息对比:")
print(f"df_rarity_long:")
print(f"  - 唯一 worker_id 数量: {df_rarity_long['worker_id'].nunique()}")
print(f"  - 唯一 country 数量: {df_rarity_long['country'].nunique()}")
print(f"  - country 值: {df_rarity_long['country'].unique()}")
print(f"  - 年份范围: {df_rarity_long['year'].min()} - {df_rarity_long['year'].max()}")

print(f"\ndf_seniority_long:")
print(f"  - 唯一 worker_id 数量: {df_seniority_long['worker_id'].nunique()}")
if 'country' in df_seniority_long.columns:
    print(f"  - 唯一 country 数量: {df_seniority_long['country'].nunique()}")
    print(f"  - country 值: {df_seniority_long['country'].unique()}")
else:
    print("  - 没有 country 列")
print(f"  - 年份范围: {df_seniority_long['year'].min()} - {df_seniority_long['year'].max()}")

# 2. 检查 worker_id 的重叠情况
rarity_workers = set(df_rarity_long['worker_id'].unique())
seniority_workers = set(df_seniority_long['worker_id'].unique())

print(f"\n2. Worker ID 重叠分析:")
print(f"  - rarity 中的 worker_id 数量: {len(rarity_workers)}")
print(f"  - seniority 中的 worker_id 数量: {len(seniority_workers)}")
print(f"  - 重叠的 worker_id 数量: {len(rarity_workers & seniority_workers)}")
print(f"  - 只在 rarity 中的 worker_id 数量: {len(rarity_workers - seniority_workers)}")
print(f"  - 只在 seniority 中的 worker_id 数量: {len(seniority_workers - rarity_workers)}")

# 3. 检查具体的不匹配记录
if len(rarity_workers - seniority_workers) > 0:
    print(f"\n只在 rarity 中的前10个 worker_id:")
    print(list(rarity_workers - seniority_workers)[:10])


=== 调查缺失 seniority 数据的原因 ===

1. 基本信息对比:
df_rarity_long:
  - 唯一 worker_id 数量: 38902
  - 唯一 country 数量: 2
  - country 值: ['India' 'US']
  - 年份范围: 1 - 10

df_seniority_long:
  - 唯一 worker_id 数量: 32628
  - 唯一 country 数量: 2
  - country 值: ['india' 'us']
  - 年份范围: 1 - 10

2. Worker ID 重叠分析:
  - rarity 中的 worker_id 数量: 38902
  - seniority 中的 worker_id 数量: 32628
  - 重叠的 worker_id 数量: 32628
  - 只在 rarity 中的 worker_id 数量: 6274
  - 只在 seniority 中的 worker_id 数量: 0

只在 rarity 中的前10个 worker_id:
['rosa-maria-6382054a', 'premanand-ramasamy-6b781933', 'benny-bligh-2b0541119', 'mike-williams-2160712a', 'kalyan-lingala-a4605a5b', 'nlkvarma', 'ishita-verma-9440105a', 'gbabu65', 'john-dixon-67139a52', 'neha-soni-4ba60a111']


In [11]:
# 更详细的缺失原因诊断 + 强化清洗与合并
from collections import Counter

# 拷贝
rar = df_rarity_long.copy()
sen = df_seniority_long.copy()

# 统一并清洗键字段
rar['worker_id'] = rar['worker_id'].astype(str).str.strip()
sen['worker_id'] = sen['worker_id'].astype(str).str.strip()
rar['year'] = rar['year'].astype(int)
sen['year'] = sen['year'].astype(int)

if 'country' in rar.columns:
    rar['country'] = rar['country'].astype(str).str.strip().str.lower()
if 'country' in sen.columns:
    sen['country'] = sen['country'].astype(str).str.strip().str.lower()

# 合并键


def get_keys(r, s):
    keys = ['worker_id', 'year']
    if 'country' in r.columns and 'country' in s.columns:
        keys = ['worker_id', 'country', 'year']
    return keys


merge_keys = get_keys(rar, sen)
print('使用合并键:', merge_keys)

# 重复键检查
dup_rar = rar.duplicated(subset=merge_keys, keep=False).sum()
dup_sen = sen.duplicated(subset=merge_keys, keep=False).sum()
print(f"rarity 重复键记录: {dup_rar}")
print(f"seniority 重复键记录: {dup_sen}")

# 键覆盖率
keys_rar = rar[merge_keys].drop_duplicates()
keys_sen = sen[merge_keys].drop_duplicates()
inter = keys_rar.merge(keys_sen, on=merge_keys, how='inner')
print(
    f"唯一键数量 -> rarity: {len(keys_rar)}, seniority: {len(keys_sen)}, 交集: {len(inter)}")

# 反连接查看未匹配
a = rar.merge(sen, on=merge_keys, how='left', indicator=True)
miss = a[a['_merge'] == 'left_only']
print(f"未匹配条数: {len(miss)} / {len(a)} ({len(miss)/len(a)*100:.2f}%)")

# 哪些 worker 缺失最多
if len(miss) > 0:
    top_workers = miss['worker_id'].value_counts().head(10)
    print('\n缺失最多的 worker_id（前10）:')
    print(top_workers)

    # 缺失分布按年份
    print('\n缺失年份分布（前10个年份）:')
    print(miss['year'].value_counts().sort_index().head(10))

    # 如果有 country，按国家
    if 'country' in merge_keys:
        print('\n缺失按国家统计:')
        print(miss['country'].value_counts())

# 进一步：检查 worker 级是否完全缺失
if len(miss) > 0:
    miss_workers = set(miss['worker_id'])
    in_sen_workers = set(sen['worker_id'])
    fully_missing = [w for w in miss_workers if w not in in_sen_workers]
    print(f"\n完全不在 seniority 的 worker 数: {len(fully_missing)}")

# 规范化后再次左连接（以 rarity 为准）
df_merged_clean = rar.merge(sen, on=merge_keys, how='left')
print('\n清洗后合并完成，缺失 seniority 数:', df_merged_clean['seniority'].isna().sum())
print('合并后形状:', df_merged_clean.shape)

# 将结果回写到主变量名
df_merged = df_merged_clean

使用合并键: ['worker_id', 'country', 'year']
rarity 重复键记录: 0
seniority 重复键记录: 0
唯一键数量 -> rarity: 389020, seniority: 326280, 交集: 326280
未匹配条数: 62740 / 389020 (16.13%)

缺失最多的 worker_id（前10）:
worker_id
%7E%7E%7Eabhishek%7E%7E%7E-lifoline-4ba22719    10
kumaran-radhakrishnan-75bb1320                  10
kumar-k-960294151                               10
kumar-b-4bb953208                               10
kumar-ananad-5b34a8204                          10
kumar-a-b76782144                               10
ksyed1                                          10
ksingh07                                        10
kshitijsanghoi                                  10
krzasteka                                       10
Name: count, dtype: int64

缺失年份分布（前10个年份）:
year
1     6274
2     6274
3     6274
4     6274
5     6274
6     6274
7     6274
8     6274
9     6274
10    6274
Name: count, dtype: int64

缺失按国家统计:
country
us       39440
india    23300
Name: count, dtype: int64

完全不在 seniority 的 worker 数: 6274

清洗后合

In [12]:
# 只保留 seniority 非 NaN 的记录
before_rows = len(df_merged)
before_users = df_merged['worker_id'].nunique()

filtered = df_merged[df_merged['seniority'].notna()].copy()
filtered = filtered.sort_values(['worker_id', 'year']).reset_index(drop=True)

after_rows = len(filtered)
after_users = filtered['worker_id'].nunique()

df_merged = filtered

print('已过滤 seniority 为 NaN 的记录')
print(f'行数: {before_rows} -> {after_rows} (去掉 {before_rows - after_rows})')
print(f'人员数: {before_users} -> {after_users} (去掉 {before_users - after_users})')
print('当前 df_merged 形状:', df_merged.shape)
df_merged.head(10)


已过滤 seniority 为 NaN 的记录
行数: 389020 -> 326280 (去掉 62740)
人员数: 38902 -> 32628 (去掉 6274)
当前 df_merged 形状: (326280, 5)


,worker_id,country,year,rarity_score,seniority
0,%C2%BBkiran-mummala-narayana-68aaba89,us,1,0.369263,Regular
1,%C2%BBkiran-mummala-narayana-68aaba89,us,2,0.384644,Regular
2,%C2%BBkiran-mummala-narayana-68aaba89,us,3,0.405297,Regular
3,%C2%BBkiran-mummala-narayana-68aaba89,us,4,0.429287,Regular
4,%C2%BBkiran-mummala-narayana-68aaba89,us,5,0.450563,Regular
5,%C2%BBkiran-mummala-narayana-68aaba89,us,6,0.470416,Regular
6,%C2%BBkiran-mummala-narayana-68aaba89,us,7,0.486217,Regular
7,%C2%BBkiran-mummala-narayana-68aaba89,us,8,0.499013,Regular
8,%C2%BBkiran-mummala-narayana-68aaba89,us,9,0.511134,Regular
9,%C2%BBkiran-mummala-narayana-68aaba89,us,10,0.522978,Regular


In [13]:
controls = pd.read_csv("/Users/lei/Library/CloudStorage/OneDrive-Nexus365/yuqi_dphil/data/occupational_segregation_paper/cleaned_data/df_final_encoded_categorical_levels.csv",
                       index_col=0)

# 清理控制变量
controls = controls.drop(columns=["organization_type", "num_followers", "industry", "company_size"], errors="ignore")

# 合并主表 + 控制变量
final_df = df_merged.merge(controls, on="worker_id", how="inner")

# 清理与转换 TODO：lack gender
# df = df[df["country"].isin(["india", "us"])].copy()
# final_df["gender"] = final_df["gender"].astype("category")
# final_df["cohort"] = final_df["cohort"].astype("category")
final_df["country"] = final_df["country"].astype("category")

# 教育分类合并
final_df["highest_educational_degree"] = final_df["highest_educational_degree"].replace(
    {"PhD": "Advanced Degree", "Master": "Advanced Degree"}
)

final_df

,worker_id,country,year,rarity_score,seniority,highest_educational_degree,whether_bachelor_university_prestigious,internationalization,highest_seniority_level,work_years,simplified_company_size
0,%C2%BBkiran-mummala-narayana-68aaba89,us,1,0.369263,Regular,Bachelor,False,Multinational,Normal,12.583,Large (500+ employees)
1,%C2%BBkiran-mummala-narayana-68aaba89,us,2,0.384644,Regular,Bachelor,False,Multinational,Normal,12.583,Large (500+ employees)
2,%C2%BBkiran-mummala-narayana-68aaba89,us,3,0.405297,Regular,Bachelor,False,Multinational,Normal,12.583,Large (500+ employees)
3,%C2%BBkiran-mummala-narayana-68aaba89,us,4,0.429287,Regular,Bachelor,False,Multinational,Normal,12.583,Large (500+ employees)
4,%C2%BBkiran-mummala-narayana-68aaba89,us,5,0.450563,Regular,Bachelor,False,Multinational,Normal,12.583,Large (500+ employees)
...,...,...,...,...,...,...,...,...,...,...,...
326275,zwheeler,us,6,0.470416,Regular,Bachelor,False,Local,Normal,13.833,Small (11-50 employees)
326276,zwheeler,us,7,0.486217,Regular,Bachelor,False,Local,Normal,13.833,Small (11-50 employees)
326277,zwheeler,us,8,0.499013,Regular,Bachelor,False,Local,Normal,13.833,Small (11-50 employees)
326278,zwheeler,us,9,0.511134,Regular,Bachelor,False,Local,Normal,13.833,Small (11-50 employees)


In [17]:
final_df = final_df.merge(df[['worker_id', 'gender']], on=["worker_id"], how="left")
final_df

,worker_id,country,year,rarity_score,seniority,highest_educational_degree,whether_bachelor_university_prestigious,internationalization,highest_seniority_level,work_years,simplified_company_size,gender
0,%C2%BBkiran-mummala-narayana-68aaba89,us,1,0.369263,Regular,Bachelor,False,Multinational,Normal,12.583,Large (500+ employees),male
1,%C2%BBkiran-mummala-narayana-68aaba89,us,2,0.384644,Regular,Bachelor,False,Multinational,Normal,12.583,Large (500+ employees),male
2,%C2%BBkiran-mummala-narayana-68aaba89,us,3,0.405297,Regular,Bachelor,False,Multinational,Normal,12.583,Large (500+ employees),male
3,%C2%BBkiran-mummala-narayana-68aaba89,us,4,0.429287,Regular,Bachelor,False,Multinational,Normal,12.583,Large (500+ employees),male
4,%C2%BBkiran-mummala-narayana-68aaba89,us,5,0.450563,Regular,Bachelor,False,Multinational,Normal,12.583,Large (500+ employees),male
...,...,...,...,...,...,...,...,...,...,...,...,...
326275,zwheeler,us,6,0.470416,Regular,Bachelor,False,Local,Normal,13.833,Small (11-50 employees),male
326276,zwheeler,us,7,0.486217,Regular,Bachelor,False,Local,Normal,13.833,Small (11-50 employees),male
326277,zwheeler,us,8,0.499013,Regular,Bachelor,False,Local,Normal,13.833,Small (11-50 employees),male
326278,zwheeler,us,9,0.511134,Regular,Bachelor,False,Local,Normal,13.833,Small (11-50 employees),male


In [20]:
# 安全删除（即便列不存在也不报错）
final_df.drop(columns=['highest_seniority_level'], errors='ignore', inplace=True)

final_df

,worker_id,country,year,rarity_score,seniority,highest_educational_degree,whether_bachelor_university_prestigious,internationalization,work_years,simplified_company_size,gender
0,%C2%BBkiran-mummala-narayana-68aaba89,us,1,0.369263,Regular,Bachelor,False,Multinational,12.583,Large (500+ employees),male
1,%C2%BBkiran-mummala-narayana-68aaba89,us,2,0.384644,Regular,Bachelor,False,Multinational,12.583,Large (500+ employees),male
2,%C2%BBkiran-mummala-narayana-68aaba89,us,3,0.405297,Regular,Bachelor,False,Multinational,12.583,Large (500+ employees),male
3,%C2%BBkiran-mummala-narayana-68aaba89,us,4,0.429287,Regular,Bachelor,False,Multinational,12.583,Large (500+ employees),male
4,%C2%BBkiran-mummala-narayana-68aaba89,us,5,0.450563,Regular,Bachelor,False,Multinational,12.583,Large (500+ employees),male
...,...,...,...,...,...,...,...,...,...,...,...
326275,zwheeler,us,6,0.470416,Regular,Bachelor,False,Local,13.833,Small (11-50 employees),male
326276,zwheeler,us,7,0.486217,Regular,Bachelor,False,Local,13.833,Small (11-50 employees),male
326277,zwheeler,us,8,0.499013,Regular,Bachelor,False,Local,13.833,Small (11-50 employees),male
326278,zwheeler,us,9,0.511134,Regular,Bachelor,False,Local,13.833,Small (11-50 employees),male


In [21]:
final_df.to_csv("final_df.csv", index=False)

In [ ]:
# 快速两向固定效应（按国家分别回归）
# - 因变量: seniority
# - 自变量: rarity_score
# - 控制: 除 worker_id, country, year, seniority, rarity_score 外的列
#   默认仅数值型控制，避免大规模独热编码导致卡顿；可改 include_categoricals=True

import pandas as pd
import numpy as np
import statsmodels.api as sm

def twoway_demean(df, cols, id_col='worker_id', time_col='year'):
    out = {}
    # 预先计算公共均值，减少重复开销
    overall_means = df[cols].mean()
    id_means = df.groupby(id_col)[cols].transform('mean')
    time_means = df.groupby(time_col)[cols].transform('mean')
    for c in cols:
        out[c] = df[c] - id_means[c] - time_means[c] + overall_means[c]
    return pd.DataFrame(out, index=df.index)

def run_fast_fe(final_df, include_categoricals=False):
    use_cols_exclude = {'worker_id','country','year','seniority','rarity_score'}
    work = final_df.copy()
    # 只保留必要列非缺失
    work = work.dropna(subset=['worker_id','year','seniority','rarity_score'])
    work['worker_id'] = work['worker_id'].astype(str).str.strip()
    work['year'] = work['year'].astype(int)

    # 控制变量集合
    control_all = [c for c in work.columns if c not in use_cols_exclude]
    if include_categoricals:
        num_controls = [c for c in control_all if np.issubdtype(work[c].dtype, np.number)]
        cat_controls = [c for c in control_all if c not in num_controls]
        # 警告：类别控制会做独热编码，可能很慢很占内存
        if cat_controls:
            dummies = pd.get_dummies(work[cat_controls], drop_first=True, dtype=float)
            work = pd.concat([work, dummies], axis=1)
            control_cols = num_controls + list(dummies.columns)
        else:
            control_cols = num_controls
    else:
        # 仅数值控制，运行最快
        control_cols = [c for c in control_all if np.issubdtype(work[c].dtype, np.number)]

    countries = pd.unique(work['country'])
    results = []
    model_objs = {}

    for ctry in countries:
        dfc = work[work['country'] == ctry].copy()
        if dfc.empty:
            continue

        cols_for_demean = ['seniority','rarity_score'] + control_cols
        dfc = dfc.dropna(subset=cols_for_demean)
        if dfc.empty:
            continue

        # 双重去均值
        dm = twoway_demean(dfc, cols_for_demean, id_col='worker_id', time_col='year')
        y_til = dm['seniority']
        X_til = dm[['rarity_score'] + control_cols]

        # OLS（不加常数；去均值后截距为0），按 worker_id 聚类稳健标准误
        ols = sm.OLS(y_til.values, X_til.values)
        res = ols.fit(cov_type='cluster', cov_kwds={'groups': dfc['worker_id'].values})

        # 取系数与SE
        # rarity_score 在 X_til 的第0列
        coef = res.params[0] if len(res.params) > 0 else np.nan
        se = res.bse[0] if len(res.bse) > 0 else np.nan
        pval = res.pvalues[0] if len(res.pvalues) > 0 else np.nan

        # Within R2（基于去均值后的回归）
        y_hat = res.fittedvalues
        ssr = np.sum((y_til.values - y_hat)**2)
        sst = np.sum((y_til.values - y_til.values.mean())**2)
        r2_within = 1 - ssr/sst if sst > 0 else np.nan

        results.append({
            'country': ctry,
            'method': 'TWFE (demean) + cluster(se worker)',
            'coef_rarity': coef,
            'se': se,
            'pval': pval,
            'R2_within': r2_within,
            'N': int(res.nobs)
        })
        model_objs[(ctry, 'twfe_demean')] = res

    return pd.DataFrame(results).sort_values('country').reset_index(drop=True), model_objs

# 运行（默认不包含类别控制，最快）
fe_fast_summary, fe_fast_models = run_fast_fe(final_df, include_categoricals=False)
print('完成。各国结果：')
print(fe_fast_summary)

# 若需要查看某个国家的完整结果（参数、聚类协方差摘要等），例如第一个国家：
if len(fe_fast_summary):
    ex_ctry = fe_fast_summary['country'].iloc[0]
    print(f'\n示例国家 {ex_ctry} 的模型摘要：')
    print(fe_fast_models[(ex_ctry, 'twfe_demean')].summary())